In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pymongo import MongoClient
import pprint

import copy
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [16]:
occupations_urls_and_names_df_mod = pd.read_csv('occupations_urls_and_names_mod.csv')

In [18]:
occupations_urls_and_names_df_mod.head()

,urls,names
0,https://famouspeople.astro-seek.com/filter_occ...,astrologers
1,https://famouspeople.astro-seek.com/filter_occ...,chemists
2,https://famouspeople.astro-seek.com/filter_occ...,cooks
3,https://famouspeople.astro-seek.com/filter_occ...,diplomats
4,https://famouspeople.astro-seek.com/filter_occ...,doctors


In [19]:
df = occupations_urls_and_names_df_mod
df.head()

,urls,names
0,https://famouspeople.astro-seek.com/filter_occ...,astrologers
1,https://famouspeople.astro-seek.com/filter_occ...,chemists
2,https://famouspeople.astro-seek.com/filter_occ...,cooks
3,https://famouspeople.astro-seek.com/filter_occ...,diplomats
4,https://famouspeople.astro-seek.com/filter_occ...,doctors


In [22]:
df.head(50)

,urls,names
0,https://famouspeople.astro-seek.com/filter_occ...,astrologers
1,https://famouspeople.astro-seek.com/filter_occ...,chemists
2,https://famouspeople.astro-seek.com/filter_occ...,cooks
3,https://famouspeople.astro-seek.com/filter_occ...,diplomats
4,https://famouspeople.astro-seek.com/filter_occ...,doctors
5,https://famouspeople.astro-seek.com/filter_occ...,entrepreneurs
6,https://famouspeople.astro-seek.com/filter_occ...,fashion-models
7,https://famouspeople.astro-seek.com/filter_occ...,journalists-publicists
8,https://famouspeople.astro-seek.com/filter_occ...,lawyers
9,https://famouspeople.astro-seek.com/filter_occ...,mathematicians


In [24]:
# occupations_url = 'https://famouspeople.astro-seek.com/filter_occupation/'
# occupations_r = requests.get(occupations_url)

#occupations_urls_and_names_df['occupations_r'] = requests.get(occupations_urls_and_names_df['urls'])
#occupations_urls_and_names_df.head()

r_dict = {}
l = 27


In [25]:
for i in range(l):
    # r = 'r_' + str(df['names'])
    
    #print('r_' + str(df['names']))
    
    s = str(df.iloc[i]['names'])
    print('r_' + s)

r_astrologers
r_chemists
r_cooks
r_diplomats
r_doctors
r_entrepreneurs
r_fashion-models
r_journalists-publicists
r_lawyers
r_mathematicians
r_philosophers
r_physicists
r_pilots-aviators
r_politicians
r_porn-stars
r_presenters
r_presidents
r_priests-theologians
r_producers
r_psychologists
r_rulers
r_scientists-inventors
r_soldiers
r_stunt-performers
r_teachers
r_the-popes
r_translators


In [28]:
client = MongoClient('localhost', 27017)
db = client.metroid
pages = db.pages

In [33]:
r_dict = {}
all_row_dict = {}

for i in range(l):
    s = str(df.iloc[i]['names'])
    r = 'r_' + s
    r_val = requests.get(df.iloc[i]['urls'])
    print(r + ' is returning ' + str(r_val))
    r_dict[r] = r_val
    pages.insert_one({'html': r_val.content})
    
    soup = BeautifulSoup(r_val.content, 'html')
    
    div = soup.find('div', {'style': 'border-left: 1px solid #E1E1E1; border-right: 1px solid #E1E1E1; width: 758px;'})
    table = div.find('table')

    rows = table.find_all('tr')
    all_rows = []
    empty_row = {'name_and_date': None, 'tag': None, 'flag': None, 'country': None}
    
    for row in rows[0::2]:
        new_row = copy.copy(empty_row)
        columns = row.find_all('td')
        new_row['name_and_date'] = columns[0].text.strip()
        new_row['tag'] = columns[1].text.strip()
        new_row['flag'] = columns[2].text.strip()
        new_row['country'] = columns[3].text.strip()
        all_rows.append(new_row)
    
    all_row_dict[r] = all_rows

r_astrologers is returning <Response [200]>
r_chemists is returning <Response [200]>
r_cooks is returning <Response [200]>
r_diplomats is returning <Response [200]>
r_doctors is returning <Response [200]>
r_entrepreneurs is returning <Response [200]>
r_fashion-models is returning <Response [200]>
r_journalists-publicists is returning <Response [200]>
r_lawyers is returning <Response [200]>
r_mathematicians is returning <Response [200]>
r_philosophers is returning <Response [200]>
r_physicists is returning <Response [200]>
r_pilots-aviators is returning <Response [200]>
r_politicians is returning <Response [200]>
r_porn-stars is returning <Response [200]>
r_presenters is returning <Response [200]>
r_presidents is returning <Response [200]>
r_priests-theologians is returning <Response [200]>
r_producers is returning <Response [200]>
r_psychologists is returning <Response [200]>
r_rulers is returning <Response [200]>
r_scientists-inventors is returning <Response [200]>
r_soldiers is retur

In [36]:
all_row_dict.keys()

dict_keys(['r_astrologers', 'r_chemists', 'r_cooks', 'r_diplomats', 'r_doctors', 'r_entrepreneurs', 'r_fashion-models', 'r_journalists-publicists', 'r_lawyers', 'r_mathematicians', 'r_philosophers', 'r_physicists', 'r_pilots-aviators', 'r_politicians', 'r_porn-stars', 'r_presenters', 'r_presidents', 'r_priests-theologians', 'r_producers', 'r_psychologists', 'r_rulers', 'r_scientists-inventors', 'r_soldiers', 'r_stunt-performers', 'r_teachers', 'r_the-popes', 'r_translators'])

### Quick Thoughts:

Not all the careers are in there yet. I think I only caught the ones in the 'occupation' section. So I'm missing the athletes and stuff. This means I have to fix the csv I imported in the beginning.

Also I'm only catching the first page of names -- so for example I'm only getting 1/5th of all the chemists if there are 5 pages of chemists.

Also it would be great to get the actual urls of each person (instead of just their name). I mean, I can guess the url based on the name, but it would be more direct to just get the urls if possible.

### Update:
I just fixed my csv. All the occupations are in there now, including athletes.

Now I will fix the page thing.

I will fix the celebrity url thing right afterward.

In [38]:
#all_row_dict

In [ ]:
https://famouspeople.astro-seek.com/filter_occupation/?narozeni_stat=&povolani=baseball-players&narozeni_stat=&pohlavi=&razeni=
https://famouspeople.astro-seek.com/filter_occupation_200/?povolani=baseball-players&pohlavi=&razeni=
https://famouspeople.astro-seek.com/filter_occupation_400/?povolani=baseball-players&pohlavi=&razeni=